In [1]:
from cavity import *

import ipywidgets as widgets
from ipywidgets import Layout, interact, FloatSlider

%matplotlib inline

In [6]:
def f(elev=44, azim=250):
    mirror = CurvedMirror(radius=1,
                          outwards_normal=np.array([1, 0, 0]),
                          origin=np.array([0, 0, 0]))
    y = np.array([-0.2, 0, 0.2])
    z = np.array([-0.2, 0, 0.2])
    Y, Z = np.meshgrid(y, z)
    X = np.zeros_like(Y)
    origin = np.stack([X, Y, Z], axis=-1)
    k_vector = np.array([1, 0, 0])
    ray = Ray(origin=origin, k_vector=k_vector)
    reflected_ray = mirror.reflect_ray(ray)
    mirror.plot()
    ax = plt.gca()
    ray.plot(ax)
    reflected_ray.plot(ax)
    ax.view_init(elev=elev, azim=azim)
    plt.show()
    

widgets.interact(f, elev=(0, 180, 1), azim=(0, 360, 1));

interactive(children=(IntSlider(value=44, description='elev', max=180), IntSlider(value=250, description='azim…

In [5]:
def f(elev=51, azim=250):
    mirror = FlatMirror(normal=np.array([1, 1, 1]), distance_from_origin=1)
    ray = Ray(origin=np.array([[0, 0, 0], [0.4, 0, -1], [0, 0.2, 0.2]]), k_vector=np.array([[1, 0, 0], [0, 1, 0], [0, 1, 0]]))
    reflected_ray = mirror.reflect_ray(ray)
    
    mirror.plot()
    ax = plt.gca()
    ray.plot(ax)
    reflected_ray.plot(ax)
    ax.view_init(elev=elev, azim=azim)
    plt.show()
    
widgets.interact(f, elev=(0, 180, 1), azim=(0, 360, 1));

interactive(children=(IntSlider(value=51, description='elev', max=180), IntSlider(value=250, description='azim…